In [1]:
from datetime import date, datetime
import math
import ROOT
import numpy as np, pandas as pd
import bokeh
import yaml
import base64
import ctypes
import logging
from bokeh.io import output_notebook, show
from bokeh.plotting import output_file
from RootInteractive.InteractiveDrawing.bokeh.bokehDrawSA import *
from RootInteractive.InteractiveDrawing.bokeh.bokehTools import bokehDrawArray
from RootInteractive.Tools.pandaTools import initMetadata

#output_notebook()

Welcome to JupyROOT 6.24/06
x bokehVisJS3DGraph.ts
x HistogramCDS.ts
Import  CDSCompress.ts
Import  DownsamplerCDS.ts
Import  CDSAlias.ts
Import  CDSJoin.ts


In [2]:
myFile = ROOT.TFile.Open("~/Software/TPC_QC_Visualization/Data/UserFiles/DownlodedFiles.root")
myFile.ls()
hist = myFile.PID
thn = hist.PID

TFile**		/home/berki/Software/TPC_QC_Visualization/Data/UserFiles/DownlodedFiles.root	
 TFile*		/home/berki/Software/TPC_QC_Visualization/Data/UserFiles/DownlodedFiles.root	
  KEY: TDirectoryFile	PID;1	PID


In [3]:
%jsroot on
c = ROOT.TCanvas("myCanvasName","The Canvas Title",800,600)

In [4]:
# Without weights
tracks = [];
coord = (ctypes.c_int*7)()
#for Bin in range(thn.GetNbins()):
for Bin in range(10000):
    val = thn.GetBinContent(Bin,coord)
    for Track in range(int(val)):
        for Dim in range(7):
            tracks.append(coord[Dim])
df = pd.DataFrame(np.reshape(tracks, newshape=(int(len(tracks)/7), 7)), 
                         columns = ['P', 'dEdxTot','dEdxMax','Phi','Tgl','Snp','NClusters'])
initMetadata(df)
#df.meta.metaData = {'P.AxisTitle': "Momentum", 'dEdxTot.AxisTitle': "dEdx"}

In [5]:
df

,P,dEdxTot,dEdxMax,Phi,Tgl,Snp,NClusters
0,1,6,9,7,9,11,28
1,1,7,11,65,5,1,4
2,3,5,9,41,3,13,32
3,3,5,9,41,3,13,32
4,3,5,9,41,3,13,32
...,...,...,...,...,...,...,...
29812,3,5,8,1,21,13,14
29813,3,5,8,1,21,13,14
29814,3,5,8,1,21,13,14
29815,3,5,8,1,21,13,14


In [6]:
bin_count = [50,50,50,72,20,20,40]
bin_min = [0,0,0,0,-1.5,-1,0]
bin_max = [10,500,100,6.3,1.5,1,152]
bin_width = [(bin_max[x]-bin_min[x])/(bin_count[x]+1) for x in range(len(bin_count))]

In [71]:
# With weights
tracks = [];
coord = (ctypes.c_int*7)()
#for Bin in range(thn.GetNbins()):
for Bin in range(10000):
    val = thn.GetBinContent(Bin,coord)
    tracks.append(val)
    for Dim in range(7):
        tracks.append(bin_min[Dim]+(coord[Dim]-0.5)*bin_width[Dim])
        #tracks.append(coord[Dim])
    
df = pd.DataFrame(np.reshape(tracks, newshape=(int(len(tracks)/8), 8)), 
                         columns = ['Weight','P', 'dEdxTot','dEdxMax','Phi','Tgl','Snp','NClusters'])
initMetadata(df)
#df.meta.metaData = {'P.AxisTitle': "Momentum", 'dEdxTot.AxisTitle': "dEdx"}
df.head()

,Weight,P,dEdxTot,dEdxMax,Phi,Tgl,Snp,NClusters
0,1.0,0.098039,53.921569,16.666667,0.560959,-0.285714,0.000000,101.951220
1,1.0,0.098039,63.725490,20.588235,5.566438,-0.857143,-0.952381,12.975610
2,5.0,0.490196,44.117647,16.666667,3.495205,-1.142857,0.190476,116.780488
3,15.0,0.294118,53.921569,16.666667,2.114384,-0.857143,0.095238,139.024390
4,2.0,0.098039,53.921569,16.666667,4.530822,-0.285714,-0.095238,31.512195


## As scatter plot

In [58]:
figureArray = [
    [['P'], ['dEdxTot'], {"colorZvar": "Weight"}],
    [['NClusters'], ['dEdxTot'], {"colorZvar": "Weight"}],
    [['Phi'], ['dEdxTot'], {"colorZvar": "Weight"}],
    [['Tgl'], ['dEdxTot'], {"colorZvar": "Weight"}],
    {"size":2, "legend_options": {"label_text_font_size": "13px"}}
]
layout = {
    "dEdx (a.u.) vs p (GeV/c)": [
        [0, {'y_visible': 1, 'x_visible':1, 'plot_height': 400}]],
    "dEdx (a.u.) vs ncls": [
        [1, {'y_visible': 1, 'x_visible':1, 'plot_height': 400}]],
    "dEdx (a.u.) vs φ (rad)": [
        [2, {'y_visible': 1, 'x_visible':1, 'plot_height': 400}]],
    "dEdx (a.u.) vs tanλ": [
        [3, {'y_visible': 1, 'x_visible':1, 'plot_height': 400}]]
}
#tooltips = [("VarA", "(@A)"), ("VarB", "(@B)"), ("VarC", "(@C)"), ("VarD", "(@D)")]
tooltips = []
pAll = bokehDrawArray(df, "P>0", figureArray, layout=layout, size=4, tooltips=tooltips)
#show(pAll[0])

In [59]:
output_file("test2.html")

widgetParams=[
    ['range', ['P']],
    ['range', ['dEdxTot']],
    ['range', ['dEdxMax']],
    ['range', ['Phi']],
    ['range', ['Tgl']],
    ['range', ['Snp']],
    ['range', ['NClusters']],
    ['range', ['Weight']]
]
widgetLayoutDesc={
    "Selection": [[0],[1],[2],[3],[4],[5],[6],[7], {'sizing_mode': 'scale_width'}]
    }    
bokehDrawSA.fromArray(df, "P>0", figureArray, widgetParams, layout=layout, tooltips=tooltips, widgetLayout=widgetLayoutDesc)


## As 1D histograms

In [93]:
histoArray = [
    {"name": "dEdxTot_hist", "variables": ["dEdxTot"], "nbins": 50, "axis": [0, 1], "weights": "Weight"},
    {"name": "NClusters_hist", "variables": ["NClusters"], "nbins": 40, "axis": [1], "weights": "Weight"},
    {"name": "Phi_hist", "variables": ["Phi"], "nbins": 72, "axis": [0, 1], "weights": "Weight"},
    {"name": "Tgl_hist", "variables": ["Tgl"], "nbins": 20, "axis": [0, 1], "weights": "Weight"}
]
figureArray = [
    [['dEdxTot'],['dEdxTot_hist'],{'visualization_type':'points'}],
    [['NClusters'],['NClusters_hist']],
    [['Phi'],['Phi_hist']],
    [['Tgl'],['Tgl_hist']]
]
figureLayoutDesc=[
    [0, 1, 2, {'commonX': 1, 'y_visible': 1, 'x_visible':1, 'plot_height': 300}],
        {'plot_height': 100, 'sizing_mode': 'scale_width'}
]
widgetParams=[['range', ['Weight']]]
tooltips = []
widgetLayoutDesc=[[0], {'sizing_mode': 'scale_width'}]

xxx = bokehDrawSA.fromArray(df, "P>0", figureArray, widgetParams, layout=figureLayoutDesc, tooltips=tooltips,
                            widgetLayout=widgetLayoutDesc, sizing_mode="scale_width", histogramArray=histoArray, palette="Plasma256")
output_file("test.html")

## As 2D histograms

In [108]:
histoArray = [
    {"name": "dEdx_vs_P", "variables": ["P", "dEdxTot"], "nbins": [50, 50], "weights": "Weight", "axis": [0, 1], "quantiles": []},
    {"name": "dEdx_vs_ncls", "variables": ["NClusters", "dEdxTot"], "nbins": [40, 50], "weights": "Weight", "axis": [0, 1], "quantiles": []},
    {"name": "dEdx_vs_phi", "variables": ["Phi", "dEdxTot"], "nbins": [72, 50], "weights": "Weight", "axis": [0, 1], "quantiles": []},
    {"name": "dEdx_vs_tgl", "variables": ["Tgl", "dEdxTot"], "nbins": [20, 50], "weights": "Weight", "axis": [0, 1], "quantiles": []}
]
figureArray = [
    [['P'], ['dEdx_vs_P'],{"visualization_type": "heatmap"}],
    [['NClusters'], ['dEdx_vs_ncls']],
    [['Phi'], ['dEdx_vs_phi']],
    [['Tgl'], ['dEdx_vs_tgl']]
]
figureLayoutDesc=[
    [[0,1],[2,3], {'y_visible': 1, 'x_visible':1, 'plot_height': 200}],
    {'plot_height': 100, 'sizing_mode': 'scale_width', 'y_visible' : 2, "size": 5}
]
widgetParams=[['range', ['Weight']]]
tooltips = []
widgetLayoutDesc=[[0], {'sizing_mode': 'scale_width'}]

xxx = bokehDrawSA.fromArray(df, "P>0", figureArray, widgetParams, layout=figureLayoutDesc, tooltips=tooltips,
                            widgetLayout=widgetLayoutDesc, sizing_mode="scale_width", histogramArray=histoArray,palette="Turbo256",colors="Bokeh",rescaleColorMapper=True)
output_file("test.html")

In [50]:
bin_count = [50,50,50,72,20,20,40]